In [9]:
# ═══════════════════════════════════════════════════════════════════════════
# ETL FIDC v1.0 - CÓDIGO COMPLETO
# ═══════════════════════════════════════════════════════════════════════════
# Copie e cole cada bloco em uma célula separada no Google Colab
# ═══════════════════════════════════════════════════════════════════════════


# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 1: IMPORTS
# ═══════════════════════════════════════════════════════════════════════════

import pandas as pd
import requests
import base64
import xml.etree.ElementTree as ET
import time
import os
from typing import Dict, List
from tqdm import tqdm

print("✅ Bibliotecas importadas!")


✅ Bibliotecas importadas!


In [10]:
# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 2: CONFIGURAÇÕES
# ═══════════════════════════════════════════════════════════════════════════

URL_API_BUSCA = "https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados"
URL_API_DOWNLOAD = "https://fnet.bmfbovespa.com.br/fnet/publico/downloadDocumento"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
}

CONFIG = {
    'delay': 2,
    'timeout_busca': 10,
    'timeout_download': 20,
    'limite_docs': 200
}

print("✅ Configurações definidas!")

✅ Configurações definidas!


In [11]:
# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 3: FUNÇÕES AUXILIARES
# ═══════════════════════════════════════════════════════════════════════════

def converter_valor(texto: str) -> float:
    if not texto or not str(texto).strip():
        return 0.0
    try:
        texto_limpo = str(texto).replace('.', '').replace(',', '.')
        return float(texto_limpo)
    except:
        return 0.0


def buscar_valor_xml(root: ET.Element, caminho: str):
    elemento = root.find(f'.//{caminho}')
    if elemento is not None and elemento.text:
        try:
            return converter_valor(elemento.text)
        except:
            return elemento.text.strip()
    return 0.0 if '/' in caminho else ''


print("✅ Funções auxiliares criadas!")

✅ Funções auxiliares criadas!


In [4]:
# ═══════════════════════════════════════════════════════════════════
# CÉLULA 4: EXTRAÇÃO XML (VERSÃO CORRIGIDA v1.0.2 FINAL)
# ═══════════════════════════════════════════════════════════════════

def extrair_dados_xml(xml_content: bytes, cnpj_fallback: str = '') -> Dict:
    """
    Extrai TODOS os campos relevantes do XML de Informe Mensal FIDC.

    CORREÇÕES v1.0.2:
    - NPL calculado CORRETAMENTE como percentual (decimal)
    - Carteira total considera ambos os tipos de crédito
    - Validação adicional para evitar valores absurdos

    Args:
        xml_content: Conteúdo XML em bytes
        cnpj_fallback: CNPJ para usar se não encontrado no XML

    Returns:
        Dict com 90+ campos estruturados
    """

    try:
        root = ET.fromstring(xml_content)
    except Exception as e:
        print(f"❌ Erro ao parsear XML: {e}")
        return {}

    dados = {
        # ═══ IDENTIFICAÇÃO ═══
        'CNPJ_FUNDO': buscar_valor_xml(root, 'NR_CNPJ_FUNDO') or cnpj_fallback,
        'DATA_COMPETENCIA': buscar_valor_xml(root, 'DT_COMPT'),

        # ═══ ATIVOS PRINCIPAIS ═══
        'ATIVO_TOTAL': buscar_valor_xml(root, 'VL_SOM_APLIC_ATIVO'),
        'DISPONIBILIDADES': buscar_valor_xml(root, 'VL_DISPONIB'),
        'CARTEIRA_TOTAL': buscar_valor_xml(root, 'VL_CARTEIRA'),

        # ═══ CRÉDITOS EXISTENTES (COM AQUISIÇÃO SUBSTANCIAL) ═══
        'CREDITOS_ADQUIRIDOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_SOM_DICRED_AQUIS'),
        'CRED_VENCIDOS_ADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_ADIMPL'),
        'CRED_VENCIDOS_INADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_INAD'),
        'CRED_INADIMPLENCIA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_INAD'),

        # ═══ DIREITOS CREDITÓRIOS (SEM AQUISIÇÃO SUBSTANCIAL) ═══
        'DICRED_TOTAL': buscar_valor_xml(root, 'DICRED/VL_DICRED'),
        'DICRED_INADIMPLENCIA': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_INAD'),

        # ═══ SEGMENTAÇÃO DA CARTEIRA ═══
        'SEGMT_INDUSTRIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_IND'),
        'SEGMT_AGRONEGOCIO': buscar_valor_xml(root, 'CART_SEGMT/VL_AGRONEG'),
        'SEGMT_ACAO_JUDICIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_ACAO_JUDIC'),

        # ═══ SEGMENTO FINANCEIRO ═══
        'SEGMT_FINANC_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_CORPOR'),

        # ═══ SETOR PÚBLICO ═══
        'SEGMT_PRECATORIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_PRECAT'),
    }

    # ─────────────────────────────────────────────────────────────────────
    # CÁLCULO DE INDICADORES (✅ VERSÃO FINAL CORRIGIDA)
    # ─────────────────────────────────────────────────────────────────────

    # 1. INADIMPLÊNCIA CONSOLIDADA (SOMA, não MAX!)
    inadimpl_cred = float(dados.get('CRED_INADIMPLENCIA', 0) or 0)
    inadimpl_dicred = float(dados.get('DICRED_INADIMPLENCIA', 0) or 0)
    dados['INADIMPLENCIA_TOTAL'] = inadimpl_cred + inadimpl_dicred

    # 2. CARTEIRA TOTAL CONSOLIDADA (ambos os tipos)
    creditos_com_aquisicao = float(dados.get('CREDITOS_ADQUIRIDOS', 0) or 0)
    creditos_sem_aquisicao = float(dados.get('DICRED_TOTAL', 0) or 0)
    carteira_total_bruta = creditos_com_aquisicao + creditos_sem_aquisicao

    # 3. ATIVO E DISPONIBILIDADES
    ativo_total = float(dados.get('ATIVO_TOTAL', 0) or 0)
    disponibilidades = float(dados.get('DISPONIBILIDADES', 0) or 0)
    carteira_liquida = float(dados.get('CARTEIRA_TOTAL', 0) or 0)

    # ─────────────────────────────────────────────────────────────────────
    # CÁLCULO DOS INDICADORES EM FORMATO DECIMAL (0.52 = 52%)
    # ─────────────────────────────────────────────────────────────────────

    # ✅ Índice NPL (Non-Performing Loans) - DECIMAL
    # Se carteira bruta > 0, calcula NPL
    # Se carteira bruta = 0 mas tem inadimplência, NPL = 100%
    # Caso contrário, NPL = 0%
    if carteira_total_bruta > 0:
        npl_calculado = dados['INADIMPLENCIA_TOTAL'] / carteira_total_bruta
        # 🔒 VALIDAÇÃO: NPL não pode ser > 100% (1.0)
        dados['INDICE_NPL_DECIMAL'] = min(npl_calculado, 1.0)
    elif dados['INADIMPLENCIA_TOTAL'] > 0:
        # Se tem inadimplência mas carteira = 0, assume 100% NPL
        dados['INDICE_NPL_DECIMAL'] = 1.0
    else:
        dados['INDICE_NPL_DECIMAL'] = 0.0

    # ✅ Taxa de liquidez - DECIMAL
    if ativo_total > 0:
        dados['TAXA_LIQUIDEZ_DECIMAL'] = disponibilidades / ativo_total
    else:
        dados['TAXA_LIQUIDEZ_DECIMAL'] = 0.0

    # ✅ Concentração em crédito - DECIMAL
    # Usa a carteira LÍQUIDA (após provisões) que está em CARTEIRA_TOTAL
    if ativo_total > 0:
        dados['CONCENTRACAO_CREDITO_DECIMAL'] = carteira_liquida / ativo_total
    else:
        dados['CONCENTRACAO_CREDITO_DECIMAL'] = 0.0

    # ─────────────────────────────────────────────────────────────────────
    # CAMPOS ADICIONAIS PARA AUDITORIA
    # ─────────────────────────────────────────────────────────────────────

    # Salvar carteira bruta para conferência
    dados['CARTEIRA_BRUTA'] = carteira_total_bruta

    # Provisões (diferença entre bruta e líquida)
    if carteira_total_bruta > 0 and carteira_liquida > 0:
        dados['PROVISOES_TOTAL'] = carteira_total_bruta - carteira_liquida
    else:
        dados['PROVISOES_TOTAL'] = 0.0

    # ─────────────────────────────────────────────────────────────────────
    # VALIDAÇÕES FINAIS
    # ─────────────────────────────────────────────────────────────────────

    # 🔒 Garantir que INDICE_NPL_DECIMAL está entre 0 e 1
    if dados['INDICE_NPL_DECIMAL'] < 0:
        dados['INDICE_NPL_DECIMAL'] = 0.0
    elif dados['INDICE_NPL_DECIMAL'] > 1:
        dados['INDICE_NPL_DECIMAL'] = 1.0

    # 🔒 Log de alerta se valores suspeitos
    if dados['INDICE_NPL_DECIMAL'] > 10:  # Se passou 10 (1000%), tem erro!
        print(f"⚠️ ALERTA: CNPJ {dados['CNPJ_FUNDO']} com NPL suspeito: {dados['INDICE_NPL_DECIMAL']}")
        print(f"   Inadimpl: {dados['INADIMPLENCIA_TOTAL']:,.2f}")
        print(f"   Carteira: {carteira_total_bruta:,.2f}")

    return dados


print("✅ Função de extração v1.0.2 FINAL criada!")
print("")
print("📊 Correções v1.0.2:")
print("   ✅ NPL calculado como DECIMAL (0.52 = 52%)")
print("   ✅ Validação: NPL entre 0% e 100%")
print("   ✅ Tratamento de carteira vazia")
print("   ✅ Conversão explícita para float")
print("   ✅ Logs de alerta para valores suspeitos")
print("")
print("🔍 Campos de auditoria:")
print("   • CARTEIRA_BRUTA (créditos totais antes de provisões)")
print("   • PROVISOES_TOTAL (diferença bruta - líquida)")
print("")


✅ Função de extração v1.0.1 criada!

📊 Correções aplicadas:
   • NPL: Soma COM + SEM aquisição
   • Carteira: Considera ambos os tipos
   • Formato: DECIMAL (0.52 = 52%)

🔍 Campos adicionais:
   • CARTEIRA_BRUTA (para auditoria)
   • PROVISOES_TOTAL (carteira bruta - líquida)

⚠️ ATENÇÃO:
   No Power BI, formate as colunas _DECIMAL como Percentual!


In [12]:
# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 5: ETL PRINCIPAL
# ═══════════════════════════════════════════════════════════════════════════

def processar_cnpj(cnpj: str) -> List[Dict]:
    resultados = []

    try:
        # Buscar documentos
        params = {
            'd': 0,
            's': 0,
            'l': CONFIG['limite_docs'],
            'cnpjFundo': cnpj
        }

        resp = requests.get(
            URL_API_BUSCA,
            params=params,
            headers=HEADERS,
            timeout=CONFIG['timeout_busca']
        )
        resp.raise_for_status()

        data = resp.json().get('data', [])
        if not data:
            return []

        df_docs = pd.DataFrame(data)

        # Filtrar informe mensal
        df_mensal = df_docs[
            (df_docs['tipoDocumento'].str.strip() == "Informe Mensal Estruturado") &
            (df_docs['situacaoDocumento'].str.strip() == "A")
        ].copy()

        if df_mensal.empty:
            return []

        # Selecionar mais recente
        df_mensal['dataRef'] = pd.to_datetime(
            df_mensal['dataReferencia'].str[3:] + '-' +
            df_mensal['dataReferencia'].str[:2] + '-01',
            format='%Y-%m-%d',
            errors='coerce'
        )

        df_mensal = df_mensal.dropna(subset=['dataRef'])
        if df_mensal.empty:
            return []

        df_mensal = df_mensal.sort_values('dataRef', ascending=False)
        doc = df_mensal.iloc[0]

        # Baixar XML
        url_download = f"{URL_API_DOWNLOAD}?id={doc['id']}"
        resp_xml = requests.get(
            url_download,
            headers=HEADERS,
            timeout=CONFIG['timeout_download']
        )
        resp_xml.raise_for_status()

        xml_content = base64.b64decode(resp_xml.content)

        # Extrair dados
        dados = extrair_dados_xml(xml_content, cnpj)
        dados['STATUS'] = 'SUCESSO'
        dados['DATA_REFERENCIA_DOC'] = doc.get('dataReferencia', '')
        dados['TIPO_COLETA'] = 'ULTIMO_MES'

        resultados.append(dados)

    except Exception as e:
        resultados.append({
            'CNPJ_FUNDO': cnpj,
            'STATUS': 'ERRO',
            'MENSAGEM_ERRO': str(e)
        })

    return resultados


print("✅ Pipeline ETL criado!")


✅ Pipeline ETL criado!


In [13]:
# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 6: UPLOAD DE DADOS (CORRIGIDO)
# ═══════════════════════════════════════════════════════════════════════════

from google.colab import files
import io

print("📤 Faça upload do arquivo: lista_cnpjs_fidc.csv")
uploaded = files.upload()

filename = list(uploaded.keys())[0]

# ✅ CORREÇÃO: Detectar automaticamente o separador
# Tentar primeiro com vírgula, depois ponto e vírgula
try:
    df_cnpjs = pd.read_csv(io.BytesIO(uploaded[filename]), sep=',')
    # Verificar se carregou corretamente
    if 'CNPJ' not in df_cnpjs.columns and len(df_cnpjs.columns) == 1:
        # Se só tem 1 coluna, o separador está errado
        raise ValueError("Separador incorreto")
except:
    # Tentar com ponto e vírgula
    df_cnpjs = pd.read_csv(io.BytesIO(uploaded[filename]), sep=';')

print(f"\n✅ Arquivo carregado: {filename}")
print(f"   • Total de CNPJs: {len(df_cnpjs):,}")
print(f"   • Colunas encontradas: {list(df_cnpjs.columns)}")

# Verificar se tem coluna CNPJ
if 'CNPJ' not in df_cnpjs.columns:
    print("\n⚠️ ATENÇÃO: Coluna 'CNPJ' não encontrada!")
    print(f"   Colunas disponíveis: {list(df_cnpjs.columns)}")
    print("\n   Por favor, verifique se o CSV tem a coluna 'CNPJ'")
else:
    print(f"\n📊 Primeiras 5 linhas:")
    print(df_cnpjs.head())
    print(f"\n✅ Pronto para processar!")


📤 Faça upload do arquivo: lista_cnpjs_fidc.csv


Saving lista_cnpjs_fidc.csv to lista_cnpjs_fidc (1).csv

✅ Arquivo carregado: lista_cnpjs_fidc (1).csv
   • Total de CNPJs: 441
   • Colunas encontradas: ['CNPJ', 'NOME_FUNDO', 'CNPJ_ORIGINAL']

📊 Primeiras 5 linhas:
             CNPJ                                         NOME_FUNDO  \
0  51199121000145  2MONEY RESP LIMITADA FIDC NP SUBORDINADA JÚNIOR 1   
1  47388724000118                   3R RESP LIMITADA FIDC NP ÚNICA 1   
2  52362629000185                                    3T FICFIDC NP 1   
3  50356663000111                        ABF VISION FIDC NP SÊNIOR 1   
4  41609997000169    ACERTO GRUPO INTER FIDC NP SUBORDINADA JÚNIOR 1   

        CNPJ_ORIGINAL  
0  51.199.121/0001-45  
1  47.388.724/0001-18  
2  52.362.629/0001-85  
3  50.356.663/0001-11  
4  41.609.997/0001-69  

✅ Pronto para processar!


In [14]:
# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 7: PROCESSAMENTO
# ═══════════════════════════════════════════════════════════════════════════

lista_cnpjs = df_cnpjs['CNPJ'].astype(str).str.zfill(14).tolist()

print(f"🚀 Processando {len(lista_cnpjs)} CNPJs...")
print(f"⏱️ Tempo estimado: ~{len(lista_cnpjs) * CONFIG['delay'] / 60:.1f} minutos\n")

todos_resultados = []

for cnpj in tqdm(lista_cnpjs, desc="Progresso"):
    resultados = processar_cnpj(cnpj)
    todos_resultados.extend(resultados)
    time.sleep(CONFIG['delay'])

df_resultado = pd.DataFrame(todos_resultados)

print(f"\n✅ Concluído!")
print(f"   Total: {len(df_resultado):,}")
print(f"   Sucesso: {(df_resultado['STATUS'] == 'SUCESSO').sum():,}")

🚀 Processando 441 CNPJs...
⏱️ Tempo estimado: ~14.7 minutos



Progresso: 100%|██████████| 441/441 [21:10<00:00,  2.88s/it]


✅ Concluído!
   Total: 441
   Sucesso: 441


In [15]:
# ═══════════════════════════════════════════════════════════════════════════
# CÉLULA 8: EXPORTAÇÃO (SEM NOTAÇÃO CIENTÍFICA!)
# ═══════════════════════════════════════════════════════════════════════════

print("💾 Exportando arquivos...\n")

# Arquivo completo
arquivo = 'fidc_dados_v1.csv'

df_resultado.to_csv(
    arquivo,
    index=False,
    encoding='utf-8-sig',
    sep=';',
    decimal=',',
    float_format='%.2f'  # ← CRÍTICO: Remove notação científica!
)

print(f"✅ {arquivo} salvo ({len(df_resultado):,} registros)")

# Download
files.download(arquivo)

print("\n🎉 PROCESSO CONCLUÍDO!")
print("\n📋 Próximos passos:")
print("   1. Importe o CSV no Power BI")
print("   2. Verifique que não há notação científica")
print("   3. Configure tipos de dados se necessário")

💾 Exportando arquivos...

✅ fidc_dados_v1.csv salvo (441 registros)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 PROCESSO CONCLUÍDO!

📋 Próximos passos:
   1. Importe o CSV no Power BI
   2. Verifique que não há notação científica
   3. Configure tipos de dados se necessário


# Task
Perform data cleaning and validation on the `fidc_dados_v1.csv` file by loading it, cleaning the 'CNPJ_FUNDO' column, converting relevant numeric and date columns, calculating and validating 'Liquidez' and 'NPL', calculating 'CARTEIRA_LIQUIDA_CALC' and 'SEM_POSICAO_FLAG', then generate a QA issues report as `qa_issues.csv` and export the final cleaned snapshot as `cleaned_snapshot.csv`.

## Load and Initial Data Cleaning

### Subtask:
Load the `fidc_dados_v1.csv` file, ensuring correct parsing with `sep=';'` and `decimal=','`. Implement a robust function to clean the 'CNPJ_FUNDO' column, removing any non-numeric characters and ensuring it's a 14-digit string. Initialize flag columns as False.


## Load and Initial Data Cleaning

This section focuses on loading the extracted FIDC data and performing initial cleaning and preparation. We will load the `fidc_dados_v1.csv` file, ensuring correct parsing for numeric values and separators. A key step involves standardizing the 'CNPJ_FUNDO' column by removing non-numeric characters and ensuring a consistent 14-digit format. Finally, we will initialize several flag columns, all set to `False`, which will be used in subsequent data validation steps.

**Reasoning**:
Now I will load the `fidc_dados_v1.csv` file, clean the 'CNPJ_FUNDO' column using a robust function to ensure a consistent 14-digit format, and initialize the specified flag columns with `False` values as requested in the subtask.



In [16]:
df_cleaned = pd.read_csv(
    'fidc_dados_v1.csv',
    sep=';',
    decimal=','
)

def clean_cnpj(cnpj_str: str) -> str:
    """
    Cleans a CNPJ string by removing non-digit characters and padding with leading zeros
    to ensure it is a 14-digit string.
    """
    if not isinstance(cnpj_str, str):
        cnpj_str = str(cnpj_str)

    # Remove all non-digit characters
    cleaned = ''.join(filter(str.isdigit, cnpj_str))

    # Pad with leading zeros if shorter than 14 characters
    return cleaned.zfill(14)

# Apply the cleaning function to the 'CNPJ_FUNDO' column
df_cleaned['CNPJ_FUNDO'] = df_cleaned['CNPJ_FUNDO'].apply(clean_cnpj)

# Initialize flag columns with False
flag_columns = [
    'ATIVO_ZERO_FLAG',
    'DIVERGE_LIQ_FLAG',
    'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG',
    'DIVERGE_NPL_FLAG',
    'SEM_POSICAO_FLAG'
]

for col in flag_columns:
    df_cleaned[col] = False

print("✅ fidc_dados_v1.csv loaded, 'CNPJ_FUNDO' cleaned, and flag columns initialized!")
print(f"   Shape of df_cleaned: {df_cleaned.shape}")
print(f"   First 5 rows of df_cleaned:\n{df_cleaned.head()}")

FileNotFoundError: [Errno 2] No such file or directory: 'fidc_dados_v1.csv'

**Reasoning**:
The previous code failed because `fidc_dados_v1.csv` was not found. Although `to_csv` was called, the file might not be accessible directly from the working directory, or there might be an issue with how the filename was used. The kernel state confirms `fidc_dados_v1.csv` exists in `/content/`. Also, to avoid potential data type issues where CNPJ might be read as a float and lose precision (as indicated by the `df_resultado` variable in the kernel state, where 'CNPJ_FUNDO' is a float in scientific notation), I will explicitly load 'CNPJ_FUNDO' as a string and adjust the `clean_cnpj` function to handle such string representations (e.g., '12345678901234.00') to correctly extract the 14-digit CNPJ.



In [17]:
df_cleaned = pd.read_csv(
    '/content/fidc_dados_v1.csv', # Use absolute path
    sep=';',
    decimal=',',
    dtype={'CNPJ_FUNDO': str} # Ensure CNPJ is read as string to preserve full value
)

def clean_cnpj(cnpj_str: str) -> str:
    """
    Cleans a CNPJ string by removing non-digit characters and ensuring it is a 14-digit string.
    It handles cases where the CNPJ might be stored as a float string (e.g., '12345678901234.00').
    """
    if not isinstance(cnpj_str, str):
        cnpj_str = str(cnpj_str)

    # Remove all non-digit characters
    cleaned = ''.join(filter(str.isdigit, cnpj_str))

    # Ensure it's 14 digits, taking the first 14 if longer (to remove '.00' suffix if present)
    # and padding with leading zeros if shorter.
    return cleaned[:14].zfill(14)

# Apply the cleaning function to the 'CNPJ_FUNDO' column
df_cleaned['CNPJ_FUNDO'] = df_cleaned['CNPJ_FUNDO'].apply(clean_cnpj)

# Initialize flag columns with False
flag_columns = [
    'ATIVO_ZERO_FLAG',
    'DIVERGE_LIQ_FLAG',
    'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG',
    'DIVERGE_NPL_FLAG',
    'SEM_POSICAO_FLAG'
]

for col in flag_columns:
    df_cleaned[col] = False

print("✅ fidc_dados_v1.csv loaded, 'CNPJ_FUNDO' cleaned, and flag columns initialized!")
print(f"   Shape of df_cleaned: {df_cleaned.shape}")
print(f"   First 5 rows of df_cleaned:\n{df_cleaned.head()}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/fidc_dados_v1.csv'

**Reasoning**:
The previous code failed because the file path `/content/fidc_dados_v1.csv` was incorrect. The file `fidc_dados_v1.csv` was saved in the current working directory, as indicated by the `arquivo` variable and the previous `to_csv` output. I will correct the file path to `fidc_dados_v1.csv`.



In [18]:
df_cleaned = pd.read_csv(
    'fidc_dados_v1.csv', # Corrected path to current directory
    sep=';',
    decimal=',',
    dtype={'CNPJ_FUNDO': str} # Ensure CNPJ is read as string to preserve full value
)

def clean_cnpj(cnpj_str: str) -> str:
    """
    Cleans a CNPJ string by removing non-digit characters and ensuring it is a 14-digit string.
    It handles cases where the CNPJ might be stored as a float string (e.g., '12345678901234.00').
    """
    if not isinstance(cnpj_str, str):
        cnpj_str = str(cnpj_str)

    # Remove all non-digit characters
    cleaned = ''.join(filter(str.isdigit, cnpj_str))

    # Ensure it's 14 digits, taking the first 14 if longer (to remove '.00' suffix if present)
    # and padding with leading zeros if shorter.
    return cleaned[:14].zfill(14)

# Apply the cleaning function to the 'CNPJ_FUNDO' column
df_cleaned['CNPJ_FUNDO'] = df_cleaned['CNPJ_FUNDO'].apply(clean_cnpj)

# Initialize flag columns with False
flag_columns = [
    'ATIVO_ZERO_FLAG',
    'DIVERGE_LIQ_FLAG',
    'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG',
    'DIVERGE_NPL_FLAG',
    'SEM_POSICAO_FLAG'
]

for col in flag_columns:
    df_cleaned[col] = False

print("✅ fidc_dados_v1.csv loaded, 'CNPJ_FUNDO' cleaned, and flag columns initialized!")
print(f"   Shape of df_cleaned: {df_cleaned.shape}")
print(f"   First 5 rows of df_cleaned:\n{df_cleaned.head()}")

FileNotFoundError: [Errno 2] No such file or directory: 'fidc_dados_v1.csv'

**Reasoning**:
The previous code failed with `FileNotFoundError` because the `fidc_dados_v1.csv` file was not found, as it was likely saved with a different name like `fidc_dados_v1 (2).csv`. I will correct the file path to `/content/fidc_dados_v1 (2).csv` to load the existing file.



In [19]:
df_cleaned = pd.read_csv(
    '/content/fidc_dados_v1 (2).csv', # Corrected path to the existing file
    sep=';',
    decimal=',',
    dtype={'CNPJ_FUNDO': str} # Ensure CNPJ is read as string to preserve full value
)

def clean_cnpj(cnpj_str: str) -> str:
    """
    Cleans a CNPJ string by removing non-digit characters and ensuring it is a 14-digit string.
    It handles cases where the CNPJ might be stored as a float string (e.g., '12345678901234.00').
    """
    if not isinstance(cnpj_str, str):
        cnpj_str = str(cnpj_str)

    # Remove all non-digit characters
    cleaned = ''.join(filter(str.isdigit, cnpj_str))

    # Ensure it's 14 digits, taking the first 14 if longer (to remove '.00' suffix if present)
    # and padding with leading zeros if shorter.
    return cleaned[:14].zfill(14)

# Apply the cleaning function to the 'CNPJ_FUNDO' column
df_cleaned['CNPJ_FUNDO'] = df_cleaned['CNPJ_FUNDO'].apply(clean_cnpj)

# Initialize flag columns with False
flag_columns = [
    'ATIVO_ZERO_FLAG',
    'DIVERGE_LIQ_FLAG',
    'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG',
    'DIVERGE_NPL_FLAG',
    'SEM_POSICAO_FLAG'
]

for col in flag_columns:
    df_cleaned[col] = False

print("✅ fidc_dados_v1 (2).csv loaded, 'CNPJ_FUNDO' cleaned, and flag columns initialized!")
print(f"   Shape of df_cleaned: {df_cleaned.shape}")
print(f"   First 5 rows of df_cleaned:\n{df_cleaned.head()}")

✅ fidc_dados_v1 (2).csv loaded, 'CNPJ_FUNDO' cleaned, and flag columns initialized!
   Shape of df_cleaned: (441, 30)
   First 5 rows of df_cleaned:
       CNPJ_FUNDO  DATA_COMPETENCIA  ATIVO_TOTAL  DISPONIBILIDADES  \
0  51199121000145               0.0   3869083.39          15471.64   
1  47388724000118               0.0   1457082.83            258.49   
2  52362629000185               0.0  12975568.17            409.85   
3  50356663000111               0.0   2373617.44         368597.06   
4  41609997000169               0.0   3902880.56            416.44   

   CARTEIRA_TOTAL  CREDITOS_ADQUIRIDOS  CRED_VENCIDOS_ADIMPLENTES  \
0      3853611.75                 0.00                       0.00   
1      1456824.34                 0.00                 1968702.86   
2     12973012.42                 0.00                       0.00   
3      1923596.49           1923596.49                 1903382.03   
4      3899837.38           3728862.17                 5193721.50   

   CRED_VENCIDO

## Convert Numeric and Date Columns

### Subtask:
Convert relevant numeric columns to float, handling PT-BR decimal parsing and '-' characters. Convert 'DATA_REFERENCIA_DOC' to a proper datetime format.


**Reasoning**:
I need to convert specified numeric columns to float and the 'DATA_REFERENCIA_DOC' column to datetime objects, handling potential errors and specific formatting as requested in the subtask.



In [20]:
numeric_cols = [
    'DATA_COMPETENCIA',
    'ATIVO_TOTAL',
    'DISPONIBILIDADES',
    'CARTEIRA_TOTAL',
    'CREDITOS_ADQUIRIDOS',
    'CRED_VENCIDOS_ADIMPLENTES',
    'CRED_VENCIDOS_INADIMPLENTES',
    'CRED_INADIMPLENCIA',
    'DICRED_TOTAL',
    'DICRED_INADIMPLENCIA',
    'SEGMT_INDUSTRIAL',
    'SEGMT_AGRONEGOCIO',
    'SEGMT_ACAO_JUDICIAL',
    'SEGMT_FINANC_CORPORATIVO',
    'SEGMT_PRECATORIOS',
    'INADIMPLENCIA_TOTAL',
    'INDICE_NPL_DECIMAL',
    'TAXA_LIQUIDEZ_DECIMAL',
    'CONCENTRACAO_CREDITO_DECIMAL',
    'CARTEIRA_BRUTA',
    'PROVISOES_TOTAL'
]

for col in numeric_cols:
    if col in df_cleaned.columns:
        # Ensure the column is treated as string before replacing, to handle mixed types safely
        df_cleaned[col] = df_cleaned[col].astype(str).str.replace('-', '0', regex=False)
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
    else:
        print(f"Column '{col}' not found in DataFrame. Skipping conversion.")

# Convert 'DATA_REFERENCIA_DOC' to datetime
# The format is 'MM/YYYY', so it should be '%m/%Y'
df_cleaned['DATA_REFERENCIA_DOC'] = pd.to_datetime(
    df_cleaned['DATA_REFERENCIA_DOC'],
    format='%m/%Y', # Expected format 'MM/YYYY'
    errors='coerce'
)

print("✅ Numeric columns converted to float and 'DATA_REFERENCIA_DOC' converted to datetime!")
print(f"   DataFrame info after conversion:")
df_cleaned.info()

✅ Numeric columns converted to float and 'DATA_REFERENCIA_DOC' converted to datetime!
   DataFrame info after conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   CNPJ_FUNDO                         441 non-null    object        
 1   DATA_COMPETENCIA                   441 non-null    float64       
 2   ATIVO_TOTAL                        441 non-null    float64       
 3   DISPONIBILIDADES                   441 non-null    float64       
 4   CARTEIRA_TOTAL                     441 non-null    float64       
 5   CREDITOS_ADQUIRIDOS                441 non-null    float64       
 6   CRED_VENCIDOS_ADIMPLENTES          441 non-null    float64       
 7   CRED_VENCIDOS_INADIMPLENTES        441 non-null    float64       
 8   CRED_INADIMPLENCIA                 441 non-null    float64

## Calculate and Validate Liquidez

### Subtask:
Calculate `liquidez_calc` as `DISPONIBILIDADES / ATIVO_TOTAL`. Apply the `ATIVO_ZERO_FLAG` where `ATIVO_TOTAL` is zero. For `ATIVO_TOTAL > 0`, validate if `round(liquidez_calc, 2)` matches `TAXA_LIQUIDEZ_DECIMAL` (after converting both to 2 decimal places), setting `DIVERGE_LIQ_FLAG` if they differ.


**Reasoning**:
I will calculate `liquidez_calc` while handling division by zero, set the `ATIVO_ZERO_FLAG` for records where `ATIVO_TOTAL` is zero, and then validate `liquidez_calc` against `TAXA_LIQUIDEZ_DECIMAL` by setting `DIVERGE_LIQ_FLAG` based on rounded values.



In [21]:
import numpy as np

# 1. Calculate liquidez_calc, handling division by zero
df_cleaned['liquidez_calc'] = np.where(
    df_cleaned['ATIVO_TOTAL'] == 0,
    0, # Set to 0 if ATIVO_TOTAL is 0, or could be np.nan
    df_cleaned['DISPONIBILIDADES'] / df_cleaned['ATIVO_TOTAL']
)

# 2. Set ATIVO_ZERO_FLAG where ATIVO_TOTAL is zero
df_cleaned.loc[df_cleaned['ATIVO_TOTAL'] == 0, 'ATIVO_ZERO_FLAG'] = True

# 3. Validate liquidez_calc against TAXA_LIQUIDEZ_DECIMAL
# Only for rows where ATIVO_TOTAL > 0
condition_for_validation = df_cleaned['ATIVO_TOTAL'] > 0

df_cleaned.loc[condition_for_validation, 'DIVERGE_LIQ_FLAG'] = (
    round(df_cleaned['liquidez_calc'], 2) !=
    round(df_cleaned['TAXA_LIQUIDEZ_DECIMAL'], 2)
).astype(bool)

print("✅ 'liquidez_calc' calculated, 'ATIVO_ZERO_FLAG' and 'DIVERGE_LIQ_FLAG' updated!")
print(f"   Number of records with ATIVO_ZERO_FLAG: {df_cleaned['ATIVO_ZERO_FLAG'].sum()}")
print(f"   Number of records with DIVERGE_LIQ_FLAG: {df_cleaned['DIVERGE_LIQ_FLAG'].sum()}")
print(f"   First 5 rows with new columns:
{df_cleaned[['ATIVO_TOTAL', 'DISPONIBILIDADES', 'TAXA_LIQUIDEZ_DECIMAL', 'liquidez_calc', 'ATIVO_ZERO_FLAG', 'DIVERGE_LIQ_FLAG']].head()}")

SyntaxError: unterminated f-string literal (detected at line 25) (ipython-input-63303973.py, line 25)

**Reasoning**:
The previous code failed due to an `unterminated f-string literal` in the print statement. I need to correct the f-string to ensure it is properly terminated by placing the entire output on a single line.



In [22]:
import numpy as np

# 1. Calculate liquidez_calc, handling division by zero
df_cleaned['liquidez_calc'] = np.where(
    df_cleaned['ATIVO_TOTAL'] == 0,
    0, # Set to 0 if ATIVO_TOTAL is 0, or could be np.nan
    df_cleaned['DISPONIBILIDADES'] / df_cleaned['ATIVO_TOTAL']
)

# 2. Set ATIVO_ZERO_FLAG where ATIVO_TOTAL is zero
df_cleaned.loc[df_cleaned['ATIVO_TOTAL'] == 0, 'ATIVO_ZERO_FLAG'] = True

# 3. Validate liquidez_calc against TAXA_LIQUIDEZ_DECIMAL
# Only for rows where ATIVO_TOTAL > 0
condition_for_validation = df_cleaned['ATIVO_TOTAL'] > 0

df_cleaned.loc[condition_for_validation, 'DIVERGE_LIQ_FLAG'] = (
    round(df_cleaned['liquidez_calc'], 2) !=
    round(df_cleaned['TAXA_LIQUIDEZ_DECIMAL'], 2)
).astype(bool)

print("✅ 'liquidez_calc' calculated, 'ATIVO_ZERO_FLAG' and 'DIVERGE_LIQ_FLAG' updated!")
print(f"   Number of records with ATIVO_ZERO_FLAG: {df_cleaned['ATIVO_ZERO_FLAG'].sum()}")
print(f"   Number of records with DIVERGE_LIQ_FLAG: {df_cleaned['DIVERGE_LIQ_FLAG'].sum()}")
print(f"   First 5 rows with new columns:\n{df_cleaned[['ATIVO_TOTAL', 'DISPONIBILIDADES', 'TAXA_LIQUIDEZ_DECIMAL', 'liquidez_calc', 'ATIVO_ZERO_FLAG', 'DIVERGE_LIQ_FLAG']].head()}")

✅ 'liquidez_calc' calculated, 'ATIVO_ZERO_FLAG' and 'DIVERGE_LIQ_FLAG' updated!
   Number of records with ATIVO_ZERO_FLAG: 3
   Number of records with DIVERGE_LIQ_FLAG: 0
   First 5 rows with new columns:
   ATIVO_TOTAL  DISPONIBILIDADES  TAXA_LIQUIDEZ_DECIMAL  liquidez_calc  \
0   3869083.39          15471.64                   0.00       0.003999   
1   1457082.83            258.49                   0.00       0.000177   
2  12975568.17            409.85                   0.00       0.000032   
3   2373617.44         368597.06                   0.16       0.155289   
4   3902880.56            416.44                   0.00       0.000107   

   ATIVO_ZERO_FLAG  DIVERGE_LIQ_FLAG  
0            False             False  
1            False             False  
2            False             False  
3            False             False  
4            False             False  


## Calculate and Validate NPL

### Subtask:
Calculate `npl_calc` as `INADIMPLENCIA_TOTAL / CARTEIRA_BRUTA`. Set `CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG` where `CARTEIRA_BRUTA` is zero and `INADIMPLENCIA_TOTAL` is greater than zero, ensuring `INDICE_NPL_DECIMAL` becomes NaN in these cases. For `CARTEIRA_BRUTA > 0`, validate if the absolute difference between `npl_calc` and `INDICE_NPL_DECIMAL` is greater than `0.01`, setting `DIVERGE_NPL_FLAG` if it is.


**Reasoning**:
I need to calculate 'npl_calc', identify and flag cases where 'CARTEIRA_BRUTA' is zero but 'INADIMPLENCIA_TOTAL' is not, set 'INDICE_NPL_DECIMAL' to NaN for these cases, and then validate 'npl_calc' against 'INDICE_NPL_DECIMAL' based on an absolute difference threshold as per the subtask instructions. Finally, I'll print summaries and display relevant columns.



In [23]:
import numpy as np

# 1. Calculate npl_calc, handling division by zero
df_cleaned['npl_calc'] = np.where(
    df_cleaned['CARTEIRA_BRUTA'] == 0,
    0, # Default to 0 if CARTEIRA_BRUTA is 0
    df_cleaned['INADIMPLENCIA_TOTAL'] / df_cleaned['CARTEIRA_BRUTA']
)

# 2. Identify rows where CARTEIRA_BRUTA is 0 AND INADIMPLENCIA_TOTAL > 0
condition_carteira_bruta_zero_com_inad = (
    (df_cleaned['CARTEIRA_BRUTA'] == 0) &
    (df_cleaned['INADIMPLENCIA_TOTAL'] > 0)
)
df_cleaned.loc[condition_carteira_bruta_zero_com_inad, 'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG'] = True

# 3. For these flagged rows, set npl_calc and INDICE_NPL_DECIMAL to NaN
df_cleaned.loc[condition_carteira_bruta_zero_com_inad, ['npl_calc', 'INDICE_NPL_DECIMAL']] = np.nan

# 4. Validate npl_calc against INDICE_NPL_DECIMAL for CARTEIRA_BRUTA > 0
# Exclude cases where CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG is True because NPL is NaN there
condition_for_npl_validation = df_cleaned['CARTEIRA_BRUTA'] > 0

df_cleaned.loc[condition_for_npl_validation, 'DIVERGE_NPL_FLAG'] = (
    np.abs(df_cleaned['npl_calc'] - df_cleaned['INDICE_NPL_DECIMAL']) > 0.01
)

# 5. Print counts
print("✅ 'npl_calc' calculated, flags updated!")
print(f"   Number of records with CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG: {df_cleaned['CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG'].sum()}")
print(f"   Number of records with DIVERGE_NPL_FLAG: {df_cleaned['DIVERGE_NPL_FLAG'].sum()}")

# 6. Display first 5 rows with relevant columns
print("\n   First 5 rows with new columns:")
print(df_cleaned[[
    'INADIMPLENCIA_TOTAL', 'CARTEIRA_BRUTA', 'INDICE_NPL_DECIMAL', 'npl_calc',
    'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG', 'DIVERGE_NPL_FLAG'
]].head())


✅ 'npl_calc' calculated, flags updated!
   Number of records with CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG: 5
   Number of records with DIVERGE_NPL_FLAG: 0

   First 5 rows with new columns:
   INADIMPLENCIA_TOTAL  CARTEIRA_BRUTA  INDICE_NPL_DECIMAL   npl_calc  \
0             95676.06      3443588.77                0.03   0.027784   
1           4648786.29       425162.92               10.93  10.934129   
2                 0.00            0.00                0.00   0.000000   
3             20214.46      1923596.49                0.01   0.010509   
4                 0.00      3728862.17                0.00   0.000000   

   CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG  DIVERGE_NPL_FLAG  
0                              False             False  
1                              False             False  
2                              False             False  
3                              False             False  
4                              False             False  


**Reasoning**:
I need to calculate 'CARTEIRA_LIQUIDA_CALC' and then set the 'SEM_POSICAO_FLAG' based on the conditions specified in the subtask.



In [24]:
import numpy as np

# 1. Calculate CARTEIRA_LIQUIDA_CALC
df_cleaned['CARTEIRA_LIQUIDA_CALC'] = df_cleaned['CREDITOS_ADQUIRIDOS'] + df_cleaned['DICRED_TOTAL']

# 2. Set SEM_POSICAO_FLAG where CARTEIRA_LIQUIDA_CALC is 0 AND ATIVO_TOTAL > 0
condition_sem_posicao = (
    (df_cleaned['CARTEIRA_LIQUIDA_CALC'] == 0) &
    (df_cleaned['ATIVO_TOTAL'] > 0)
)
df_cleaned.loc[condition_sem_posicao, 'SEM_POSICAO_FLAG'] = True

print("✅ 'CARTEIRA_LIQUIDA_CALC' calculated and 'SEM_POSICAO_FLAG' updated!")
print(f"   Number of records with SEM_POSICAO_FLAG: {df_cleaned['SEM_POSICAO_FLAG'].sum()}")
print("   First 5 rows with new columns:")
print(df_cleaned[[
    'CREDITOS_ADQUIRIDOS', 'DICRED_TOTAL', 'CARTEIRA_LIQUIDA_CALC',
    'ATIVO_TOTAL', 'SEM_POSICAO_FLAG'
]].head())


✅ 'CARTEIRA_LIQUIDA_CALC' calculated and 'SEM_POSICAO_FLAG' updated!
   Number of records with SEM_POSICAO_FLAG: 46
   First 5 rows with new columns:
   CREDITOS_ADQUIRIDOS  DICRED_TOTAL  CARTEIRA_LIQUIDA_CALC  ATIVO_TOTAL  \
0                 0.00    3443588.77             3443588.77   3869083.39   
1                 0.00     425162.92              425162.92   1457082.83   
2                 0.00          0.00                   0.00  12975568.17   
3           1923596.49          0.00             1923596.49   2373617.44   
4           3728862.17          0.00             3728862.17   3902880.56   

   SEM_POSICAO_FLAG  
0             False  
1             False  
2              True  
3             False  
4             False  


## Generate QA Issues Report

### Subtask:
Filter the main DataFrame to include only rows where any of the validation flags (`ATIVO_ZERO_FLAG`, `DIVERGE_LIQ_FLAG`, `CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG`, `DIVERGE_NPL_FLAG`, `SEM_POSICAO_FLAG`) are True. Export this filtered DataFrame to `qa_issues.csv`.


**Reasoning**:
I need to filter the `df_cleaned` DataFrame based on the validation flags, create a `df_qa_issues` DataFrame, and then export it to a CSV file with the specified formatting.



In [25]:
import pandas as pd

# 1. Define a list of validation flag columns
validation_flags = [
    'ATIVO_ZERO_FLAG',
    'DIVERGE_LIQ_FLAG',
    'CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG',
    'DIVERGE_NPL_FLAG',
    'SEM_POSICAO_FLAG'
]

# 2. Create a boolean Series for rows with any True flag
has_issues = df_cleaned[validation_flags].any(axis=1)

# 3. Filter df_cleaned to create df_qa_issues
df_qa_issues = df_cleaned[has_issues].copy()

# 4. Export df_qa_issues to 'qa_issues.csv'
qa_issues_filename = 'qa_issues.csv'
df_qa_issues.to_csv(
    qa_issues_filename,
    sep=';',
    decimal=',',
    index=False,
    encoding='utf-8-sig',
    float_format='%.2f'
)

print(f"✅ QA issues report generated: {qa_issues_filename}")
print(f"   Number of rows with issues: {len(df_qa_issues)}")
print(f"   First 5 rows of QA issues:\n{df_qa_issues.head()}")

✅ QA issues report generated: qa_issues.csv
   Number of rows with issues: 49
   First 5 rows of QA issues:
        CNPJ_FUNDO  DATA_COMPETENCIA   ATIVO_TOTAL  DISPONIBILIDADES  \
2   52362629000185               0.0  1.297557e+07            409.85   
36  35689357000103               0.0  1.422617e+08              0.00   
46  48960844000100               0.0  1.415174e+05            100.00   
54  18048642000126               0.0  2.428103e+09              0.00   
57  34218194000118               0.0  3.630317e+07            100.00   

    CARTEIRA_TOTAL  CREDITOS_ADQUIRIDOS  CRED_VENCIDOS_ADIMPLENTES  \
2     1.297301e+07                  0.0                        0.0   
36    1.422600e+08                  0.0                        0.0   
46    1.414124e+05                  0.0                        0.0   
54    2.428103e+09                  0.0                        0.0   
57    7.415000e+01                  0.0                        0.0   

    CRED_VENCIDOS_INADIMPLENTES  CRED_

## Export Cleaned Snapshot

### Subtask:
Export the fully cleaned and validated DataFrame (including all calculated columns and flags) to `cleaned_snapshot.csv`, ensuring numeric columns are properly formatted without scientific notation and using `;` as separator and `,` as decimal point.


**Reasoning**:
I need to export the `df_cleaned` DataFrame to a CSV file named `cleaned_snapshot.csv` with the specified formatting (no index, semicolon separator, comma decimal point, two decimal places for floats, and 'utf-8-sig' encoding).



In [26]:
cleaned_snapshot_filename = 'cleaned_snapshot.csv'
df_cleaned.to_csv(
    cleaned_snapshot_filename,
    index=False,
    sep=';',
    decimal=',',
    float_format='%.2f',
    encoding='utf-8-sig'
)

print(f"✅ Cleaned snapshot exported to '{cleaned_snapshot_filename}'")
print(f"   Shape of exported data: {df_cleaned.shape}")

✅ Cleaned snapshot exported to 'cleaned_snapshot.csv'
   Shape of exported data: (441, 33)


## Define Optional Diff Function

### Subtask:
Create a Python function that accepts two DataFrames (representing two versions of the data). This function will compare numerical columns between the two DataFrames for common CNPJs and identify differences, outputting a `diff_v1_v2.csv` file with the CNPJ, column name, and numeric difference. This function will only be executed if the user explicitly requests it.


## Define Optional Diff Function

### Subtask:
Create a Python function that accepts two DataFrames (representing two versions of the data). This function will compare numerical columns between the two DataFrames for common CNPJs and identify differences, outputting a `diff_v1_v2.csv` file with the CNPJ, column name, and numeric difference. This function will only be executed if the user explicitly requests it.

#### Instructions
1. Define a function, for example, `generate_diff_report`, that takes two pandas DataFrames as input, representing the original and new versions of the data.
2. Inside the function, identify common CNPJs between the two DataFrames.
3. Iterate through the numerical columns present in both DataFrames.
4. For each common CNPJ and numerical column, calculate the difference between the values in the two DataFrames.
5. Store these differences along with the CNPJ and column name in a new DataFrame.
6. Export this new DataFrame to a CSV file named `diff_v1_v2.csv`, using `;` as a separator, `,` as a decimal point, and formatting float values to two decimal places.

**Reasoning**:
This function, `generate_diff_report`, will be crucial for performing quality assurance by comparing the newly processed `cleaned_snapshot.csv` with a previous version (`fidc_dados_v1.csv` is used here as a placeholder for a 'previous version' for demonstration, although in a real scenario it would be a distinct 'v1' to compare with a 'v2'). This allows us to quickly identify any discrepancies in numerical values, which is essential for maintaining data integrity and ensuring that transformations are applied correctly. The report will highlight differences by CNPJ and column, facilitating targeted investigation of data changes.

## Define Optional Diff Function

### Subtask:
Create a Python function that accepts two DataFrames (representing two versions of the data). This function will compare numerical columns between the two DataFrames for common CNPJs and identify differences, outputting a `diff_v1_v2.csv` file with the CNPJ, column name, and numeric difference. This function will only be executed if the user explicitly requests it.

#### Instructions
1. Define a function, for example, `generate_diff_report`, that takes two pandas DataFrames as input, representing the original and new versions of the data.
2. Inside the function, identify common CNPJs between the two DataFrames.
3. Iterate through the numerical columns present in both DataFrames.
4. For each common CNPJ and numerical column, calculate the difference between the values in the two DataFrames.
5. Store these differences along with the CNPJ and column name in a new DataFrame.
6. Export this new DataFrame to a CSV file named `diff_v1_v2.csv`, using `;` as a separator, `,` as a decimal point, and formatting float values to two decimal places.

**Reasoning**:
This function, `generate_diff_report`, will be crucial for performing quality assurance by comparing the newly processed `cleaned_snapshot.csv` with a previous version (`fidc_dados_v1.csv` is used here as a placeholder for a 'previous version' for demonstration, although in a real scenario it would be a distinct 'v1' to compare with a 'v2'). This allows us to quickly identify any discrepancies in numerical values, which is essential for maintaining data integrity and ensuring that transformations are applied correctly. The report will highlight differences by CNPJ and column, facilitating targeted investigation of data changes.

## Define Optional Diff Function

### Subtask:
Create a Python function that accepts two DataFrames (representing two versions of the data). This function will compare numerical columns between the two DataFrames for common CNPJs and identify differences, outputting a `diff_v1_v2.csv` file with the CNPJ, column name, and numeric difference. This function will only be executed if the user explicitly requests it.

#### Instructions
1. Define a function, for example, `generate_diff_report`, that takes two pandas DataFrames as input, representing the original and new versions of the data.
2. Inside the function, identify common CNPJs between the two DataFrames.
3. Iterate through the numerical columns present in both DataFrames.
4. For each common CNPJ and numerical column, calculate the difference between the values in the two DataFrames.
5. Store these differences along with the CNPJ and column name in a new DataFrame.
6. Export this new DataFrame to a CSV file named `diff_v1_v2.csv`, using `;` as a separator, `,` as a decimal point, and formatting float values to two decimal places.

**Reasoning**:
This function, `generate_diff_report`, will be crucial for performing quality assurance by comparing the newly processed `cleaned_snapshot.csv` with a previous version (`fidc_dados_v1.csv` is used here as a placeholder for a 'previous version' for demonstration, although in a real scenario it would be a distinct 'v1' to compare with a 'v2'). This allows us to quickly identify any discrepancies in numerical values, which is essential for maintaining data integrity and ensuring that transformations are applied correctly. The report will highlight differences by CNPJ and column, facilitating targeted investigation of data changes.

**Reasoning**:
I need to provide the actual Python code for the `generate_diff_report` function as described in the previous markdown block. This function will take two DataFrames, find common CNPJs, compare numerical columns, and export the differences to a CSV file.



In [27]:
def generate_diff_report(df_v1: pd.DataFrame, df_v2: pd.DataFrame) -> None:
    """
    Compares numerical columns between two DataFrames (df_v1 and df_v2) for common CNPJs,
    identifies differences, and outputs a CSV file with the CNPJ, column name, and numeric difference.

    Args:
        df_v1: The first DataFrame (e.g., original data).
        df_v2: The second DataFrame (e.g., new/cleaned data).
    """

    # Ensure CNPJ_FUNDO is the index for easier alignment
    df_v1_indexed = df_v1.set_index('CNPJ_FUNDO')
    df_v2_indexed = df_v2.set_index('CNPJ_FUNDO')

    # Identify common CNPJs
    common_cnpjs = list(set(df_v1_indexed.index) & set(df_v2_indexed.index))

    if not common_cnpjs:
        print("No common CNPJs found between the two DataFrames. No diff report generated.")
        return

    # Filter to common CNPJs
    df_v1_common = df_v1_indexed.loc[common_cnpjs]
    df_v2_common = df_v2_indexed.loc[common_cnpjs]

    # Identify common numerical columns
    numeric_cols_v1 = df_v1_common.select_dtypes(include=np.number).columns
    numeric_cols_v2 = df_v2_common.select_dtypes(include=np.number).columns
    common_numeric_cols = list(set(numeric_cols_v1) & set(numeric_cols_v2))

    if not common_numeric_cols:
        print("No common numerical columns found between the two DataFrames. No diff report generated.")
        return

    diff_records = []

    for cnpj in common_cnpjs:
        for col in common_numeric_cols:
            val_v1 = df_v1_common.loc[cnpj, col]
            val_v2 = df_v2_common.loc[cnpj, col]

            # Handle potential NaN values, consider them as 0 for difference calculation if applicable
            # Or exclude if any is NaN
            if pd.isna(val_v1) or pd.isna(val_v2):
                if not pd.isna(val_v1) or not pd.isna(val_v2): # If only one is NaN, it's a difference
                    diff = val_v2 - val_v1 if not pd.isna(val_v1) else val_v2 # If val_v1 is NaN, difference is val_v2
                    if not pd.isna(val_v2): # If val_v2 is NaN, difference is -val_v1
                        diff = val_v2 if pd.isna(val_v1) else -val_v1
                    else:
                        diff = np.nan
                else:
                    diff = 0.0 # Both NaN, no difference
            else:
                diff = val_v2 - val_v1

            if diff != 0:
                diff_records.append({
                    'CNPJ_FUNDO': cnpj,
                    'COLUNA': col,
                    'DIFERENCA': diff
                })

    if diff_records:
        df_diff = pd.DataFrame(diff_records)
        diff_filename = 'diff_v1_v2.csv'
        df_diff.to_csv(
            diff_filename,
            index=False,
            sep=';',
            decimal=',',
            float_format='%.2f',
            encoding='utf-8-sig'
        )
        print(f"✅ Difference report generated: {diff_filename} ({len(df_diff)} differences found).")
    else:
        print("No numerical differences found between the two DataFrames.")

print("✅ `generate_diff_report` function defined!")

✅ `generate_diff_report` function defined!


## Final Task

### Subtask:
Summarize the performed actions, the generated output files (`qa_issues.csv` and `cleaned_snapshot.csv`), and explain how to use the optional diff function. Confirm that the script now adheres to all specified audit observations.


## Summary:

### Data Analysis Key Findings

*   **Initial Data Preparation**: The `fidc_dados_v1 (2).csv` file was successfully loaded. The 'CNPJ\_FUNDO' column was cleaned to a standardized 14-digit string format.
*   **Data Type Conversion**: 21 numeric columns were successfully converted to `float64`, handling specific parsing rules (e.g., replacing '-' with '0'). The 'DATA\_REFERENCIA\_DOC' column was converted to a `datetime64[ns]` format.
*   **Liquidez Calculation and Validation**:
    *   `liquidez_calc` was computed based on 'DISPONIBILIDADES' and 'ATIVO\_TOTAL'.
    *   The `ATIVO_ZERO_FLAG` was set to `True` for 3 records where 'ATIVO\_TOTAL' was zero.
    *   No discrepancies were found (0 records flagged) for `DIVERGE_LIQ_FLAG` when comparing `liquidez_calc` with `TAXA_LIQUIDEZ_DECIMAL` after rounding to two decimal places.
*   **NPL Calculation and Validation**:
    *   `npl_calc` was computed from 'INADIMPLENCIA\_TOTAL' and 'CARTEIRA\_BRUTA'.
    *   The `CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG` was set to `True` for 5 records where 'CARTEIRA\_BRUTA' was zero but 'INADIMPLENCIA\_TOTAL' was greater than zero; for these, `npl_calc` and `INDICE_NPL_DECIMAL` were set to `NaN`.
    *   No discrepancies were found (0 records flagged) for `DIVERGE_NPL_FLAG` based on an absolute difference threshold of 0.01 between `npl_calc` and `INDICE_NPL_DECIMAL`.
*   **Portfolio Liquidity Flag**:
    *   `CARTEIRA_LIQUIDA_CALC` was calculated as the sum of 'CREDITOS\_ADQUIRIDOS' and 'DICRED\_TOTAL'.
    *   The `SEM_POSICAO_FLAG` was set to `True` for 46 records where `CARTEIRA_LIQUIDA_CALC` was zero and 'ATIVO\_TOTAL' was greater than zero.
*   **QA Issues Report**: A `qa_issues.csv` file was generated, containing 49 rows that exhibited at least one of the defined data quality issues (i.e., any validation flag was `True`).
*   **Cleaned Data Export**: A `cleaned_snapshot.csv` file was successfully exported, containing the fully processed data (441 rows and 33 columns) with all new calculations and flags, using specified formatting for separators and decimal points.
*   **Optional Diff Function**: A Python function, `generate_diff_report`, was defined to compare numerical columns between two DataFrames based on common 'CNPJ\_FUNDO' values and report any differences to `diff_v1_v2.csv`.

### Insights or Next Steps

*   The `qa_issues.csv` report (containing 49 records) should be thoroughly reviewed to understand the nature and impact of the identified data anomalies. Prioritize investigation into the 46 records flagged with `SEM_POSICAO_FLAG` and the 5 records with `CARTEIRA_BRUTA_ZERO_COM_INAD_FLAG` as these represent potential data inconsistencies or unusual financial positions.
*   Utilize the `generate_diff_report` function in future data processing cycles to quickly identify changes in numerical values between different versions of the cleaned data, which will be crucial for auditability and quality control, especially for changes beyond the initially defined flags.
